In [1]:
import re, os, sys, unicodedata
import pandas as pd
import numpy as np
import py_stringmatching as sm
import codecs

def _normalize(data):
    """ Normalize unicode data in Python to remove umlauts, accents, spaces, etc."""
    normal = unicodedata.normalize('NFKD', data).encode('ASCII', 'ignore')
    val = normal.decode("utf-8")
    val = val.lower()
    # remove special characters
    val = re.sub('[^A-Za-z]+', '', val)
    return val

In [ ]:
BOUNDARY = len(df0)

In [2]:
path = '/Users/galvanize/infosys/cip/data/nc_data'
filename0, filename1 = 'nc0.csv', 'nc1.csv'
filepath0, filepath1 = os.path.join(path, filename0), os.path.join(path, filename1)
df0 = pd.read_csv(filepath0, dtype={'recid':np.int64, 'givenname':str, 'surname':str, 'suburb':str, 'postcode':str})
df1 = pd.read_csv(filepath1, dtype={'recid':np.int64, 'givenname':str, 'surname':str, 'suburb':str, 'postcode':str})
ROW_BOUNDARY = len(df0) #separation of first and second files
df = pd.concat([df0, df1]).reset_index()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 6 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   index      int64 
 1   recid      int64 
 2   givenname  object
 3   surname    object
 4   suburb     object
 5   postcode   object
dtypes: int64(2), object(4)
memory usage: 91.6+ MB


In [17]:
ROW_BOUNDARY = len(df0)
ROW_BOUNDARY

1000000

In [3]:
answers = df['recid']
df = df.drop(['recid', 'index'], axis=1)

In [4]:
for col in df.columns:
    df.loc[:,col] = df[col].apply(lambda x: str(x)).str.replace('[^A-Za-z0-9]+', '', regex=True)

In [5]:
tok = sm.QgramTokenizer(qval=3, return_set=True, padding=False)

In [6]:
for col in df.columns:
    df.loc[:,f'{col}_qgrams'] = df[col].apply(lambda x: set(tok.tokenize(x)))
df.head()

,givenname,surname,suburb,postcode,givenname_qgrams,surname_qgrams,suburb_qgrams,postcode_qgrams
0,kadelyn,gragnani,waxhaw,2873,"{lyn, kad, ely, del, ade}","{agn, nan, ani, rag, gna, gra}","{haw, axh, xha, wax}","{287, 873}"
1,ronel,carter,washington,2788g,"{ron, nel, one}","{art, rte, car, ter}","{ngt, hin, shi, ash, was, gto, ton, ing}","{788, 278, 88g}"
2,jason,roehrig,millersceeek,286s1,"{jas, aso, son}","{ehr, roe, rig, hri, oeh}","{sce, cee, rsc, eee, ler, mil, ers, lle, ill, ...","{86s, 286, 6s1}"
3,antrea,mure,greensboro,27410,"{rea, tre, ntr, ant}","{mur, ure}","{ree, sbo, een, bor, ens, oro, gre, nsb}","{410, 741, 274}"
4,caedar,oliver,jacksonville,28542,"{eda, dar, aed, cae}","{liv, oli, ver, ive}","{jac, cks, vil, ack, onv, lle, ill, nvi, kso, ...","{542, 285, 854}"


In [7]:
all_suburb_qgrams = set().union(*df['suburb_qgrams'].values)
all_givenname_qgrams = set().union(*df['givenname_qgrams'].values)
all_surname_qgrams = set().union(*df['surname_qgrams'].values)
all_postcode_qgrams = set().union(*df['postcode_qgrams'].values)
len(all_suburb_qgrams), len(all_givenname_qgrams), len(all_surname_qgrams), len(all_postcode_qgrams)

(12536, 13470, 16608, 2199)

In [8]:
from collections import defaultdict

df.head()

In [9]:
postcode_dict = dict(zip(df.index, df.postcode_qgrams))
postcode_dict[0]

{'287', '873'}

In [10]:
inverted_postcode_dict = defaultdict(list)
for key, values in postcode_dict.items():
    for value in values:
        inverted_postcode_dict[value].append(key)


inverted_postcode_dict['287'] = 
[0,
 46,
 53,
 75,
 84,
 85,
 123,
 127,
 156,
 166,
 ...

for each key, e.g., '287', the list of values are the blocks I need to compare!

ensure that there are no duplicates in the groupings from one key to the next

In [21]:
from itertools import combinations
from more_itertools import partition

In [25]:
_, item287 = partition(lambda t: t[0] < ROW_BOUNDARY and t[1] >= ROW_BOUNDARY, combinations(inverted_postcode_dict['287'], 2))

In [26]:
next(item287)

(0, 1000005)

In [27]:
df.iloc[[0,100005]]

,givenname,surname,suburb,postcode,givenname_qgrams,surname_qgrams,suburb_qgrams,postcode_qgrams
0,kadelyn,gragnani,waxhaw,2873,"{lyn, kad, ely, del, ade}","{agn, nan, ani, rag, gna, gra}","{haw, axh, xha, wax}","{287, 873}"
100005,grover,turner,lenoir,28645,"{gro, rov, ove, ver}","{rne, urn, ner, tur}","{eno, noi, len, oir}","{286, 864, 645}"


In [ ]:
combinations(inverted_postcode_dict['287'], 2) #returns an iterator which ultimately returns tuples

In [ ]:
postcode_df = pd.DataFrame(data=all_postcode_qgrams, columns=['postcode_grams'])
postcode_df

total = set()
f = [{4,5}, {5,6}, {6,8}, {9,4}]
for item in f:
    total.update(item)

total

task: 
    find all the existing trigrams for each field


In [ ]:
all_suburb_qgrams = set().union(*df['suburb0_qgrams'].values).union(*df['suburb1_qgrams'])
all_givenname_qgrams = set().union(*df['givenname0_qgrams'].values).union(*df['givenname1_qgrams'])
all_surname_qgrams = set().union(*df['surname0_qgrams'].values).union(*df['surname1_qgrams'])
all_postcode_qgrams = set().union(*df['postcode0_qgrams'].values).union(*df['postcode1_qgrams'])
len(all_suburb_qgrams), len(all_givenname_qgrams), len(all_surname_qgrams), len(all_postcode_qgrams)

In [ ]:
test_df = df.iloc[:10]
test_df

In [ ]:
#let's concentrate on the postcodes

In [ ]:
postcode_qgrams = set().union(*test_df['postcode0_qgrams'].values).union(*test_df['postcode1_qgrams'])
len(postcode_qgrams)

In [ ]:
test_df

Old Version

In [ ]:
task: 
    for each column 
    assign the column index to trigrams it contains 

## super slow!!!!
# suburbs key = trigram, value = index

suburb_dd = defaultdict(list)
for value, keys in zip(df.index, df['suburb0_qgrams']):
    for key in keys:
        suburb_dd[key].append(['0',value])

for value, keys in zip(df.index, df['suburb1_qgrams']):
    for key in keys:
        suburb_dd[key].append(['1',value])

givenname_dd =  defaultdict(list)
for value, keys in zip(df.index, df['givenname0_qgrams']):
    for key in keys:
        givenname_dd[key].append(['0',value])

for value, keys in zip(df.index, df['givenname1_qgrams']):
    for key in keys:
        givenname_dd[key].append(['1',value])

surname_dd = defaultdict(list)
for value, keys in zip(df.index, df['surname0_qgrams']):
    for key in keys:
        surname_dd[key].append(['0',value])

for value, keys in zip(df.index, df['surname1_qgrams']):
    for key in keys:
        surname_dd[key].append(['1',value])

postcode_dd = defaultdict(list)
for value, keys in zip(df.index, df['postcode0_qgrams']):
    for key in keys:
        postcode_dd[key].append(['0',value])

for value, keys in zip(df.index, df['postcode1_qgrams']):
    for key in keys:
        postcode_dd[key].append(['1',value])

In [ ]:
import pickle

In [ ]:
saved_data = {}
saved_data['suburb_dd']= suburb_dd
saved_data['givenname_dd']= givenname_dd
saved_data['surname_dd']= surname_dd
saved_data['postcode_dd']= postcode_dd

saved_data['all_suburb_qgrams']= all_suburb_qgrams
saved_data['all_givenname_qgrams']= all_givenname_qgrams
saved_data['all_surname_qgrams']= all_surname_qgrams
saved_data['all_postcode_qgrams']= all_postcode_qgrams

saved_data['df']= df
saved_data['df0']= df0
saved_data['df1']= df1

with open('nc_test.pickle', 'wb') as f:
    pickle.dump(saved_data, f)

In [ ]:
computed_items = {}
computed_items['suburb_dd']= suburb_dd
computed_items['givenname_dd']= givenname_dd
computed_items['surname_dd']= surname_dd
computed_items['postcode_dd']= postcode_dd

computed_items['all_suburb_qgrams']= all_suburb_qgrams
computed_items['all_givenname_qgrams']= all_givenname_qgrams
computed_items['all_surname_qgrams']= all_surname_qgrams
computed_items['all_postcode_qgrams']= all_postcode_qgrams

with open('nc_test_computed.pickle', 'wb') as f:
    pickle.dump(computed_items, f)
    
data_frame = {}
data_frame['df']= df
with open('nc_test_df.pickle', 'wb') as f:
    pickle.dump(data_frame, f)

In [ ]:
len(all_suburb_qgrams)

In [ ]:
# SLOW. don't run this
# for key in suburb_dd.keys():
#     print(key, suburb_dd[key])

test_key = 'wax'
print(len(suburb_dd[test_key]))
# print(suburb_dd[test_key])
# make a tuple for each suburb. 
# first item is the dataset #,
# second item is the key,
# third item is a set of indexes for that key

test_key = suburb_dd[test_key]

In [ ]:
test_key_0 = 

# assign trigrams for each r
dd = defaultdict(list)


QGRAMS = namedtuple('QGRAMS','suburb, givenname, surname, postcode') 

data0 = QGRAMS(df['suburb0_qgrams'],df['givenname0_qgrams'],df['surname0_qgrams'],df['postcode0_qgrams'])

data1 = QGRAMS(df['suburb1_qgrams'],df['givenname1_qgrams'],df['surname1_qgrams'],df['postcode1_qgrams'])

data0.suburb

data1.suburb